In [1]:
# Import modules
from pyspark.ml import Pipeline
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.feature import VectorAssembler, StringIndexer, IndexToString, SQLTransformer
from pyspark.ml.classification import *
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType, BooleanType, DoubleType
from pyspark.ml.tuning import  ParamGridBuilder
from pyspark.ml.tuning import  CrossValidator

In [2]:
# Define input path and constants
inputData = "/data/students/bigdata-01QYD/Lab9_DBD/Reviews.csv"

In [3]:
# Load the data
# Create a DataFrame from Reviews.csv
reviews = spark.read.load(inputData,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)

In [4]:
reviews.printSchema()
#reviews.show()

root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: integer (nullable = true)
 |-- HelpfulnessDenominator: integer (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Time: integer (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



In [5]:
# Select only the records with HelpfulnessDenominator>0 (i.e., rated reviews)
reviewsWithVotes = reviews.filter("HelpfulnessDenominator>0")

In [6]:
# Create and compute the value of Column label for the selected rated reviews
def labelAttribute(HelpfulnessNumerator, HelpfulnessDenominator):
    if HelpfulnessNumerator/HelpfulnessDenominator>0.9:
        return 1.0
    else:
        return 0.0
    
    
spark.udf.register("labelAttribute", labelAttribute, DoubleType())    

<function __main__.labelAttribute(HelpfulnessNumerator, HelpfulnessDenominator)>

In [7]:
# Create the class attribute
# For this task, a review belongs to the “useful” class if its helpfulness index is above 90% (0.9).
reviewsLabelWithVotes = reviewsWithVotes\
.selectExpr("*", "labelAttribute(HelpfulnessNumerator, HelpfulnessDenominator) as label")

In [8]:
#reviewsLabelWithVotes.printSchema()
#reviewsLabelWithVotes.select("HelpfulnessNumerator", "HelpfulnessDenominator", "label").show()

In [9]:
# Split the dataframe with Column label in training and test set
(reviews_train, reviews_test) = reviewsLabelWithVotes.randomSplit([0.75,0.25], seed=10)

In [10]:
# Create/Define the preprocessing steps and the classification algorithm you want to use 
# and the content of the pipeline that is used to train the model on reviews_train and apply it on reviews_test

In [11]:
# In this solution we decided to use
# - The length of text
# - The number of words in text
# - The length of summary
# - The number of words in summary
# - The number of ! appearing in text
# - The number of ! appearing in  summary
# - The score assigned to the reviewed item in this review

In [12]:
# Define the function that is used to compute "The length of text"
spark.udf.register("lenText", lambda text: len(text), IntegerType())

<function __main__.<lambda>(text)>

In [13]:
# Define the function that is used to compute "The number of words in text"
spark.udf.register("numWordsText", lambda text: len(text.split(" ")), IntegerType())

<function __main__.<lambda>(text)>

In [14]:
# Define the function that is used to compute "The length of summary"
spark.udf.register("lenSummary", lambda summary: len(summary), IntegerType())

<function __main__.<lambda>(summary)>

In [15]:
# Define the function that is used to compute "The number of words in summary"
spark.udf.register("numWordsSummary", lambda summary: len(summary.split(" ")), IntegerType())

<function __main__.<lambda>(summary)>

In [16]:
# Define the function that is used to compute "The number of ! appearing in text"
spark.udf.register("numExclMark", lambda text: len(text.split("!")), IntegerType())

<function __main__.<lambda>(text)>

In [17]:
# Define the function that is used to compute "The number of ! appearing in summary"
spark.udf.register("numExclMarkSummary", lambda summary: len(summary.split("!")), IntegerType())

<function __main__.<lambda>(summary)>

In [18]:
# Define an SQLTranformer to create the columns we are interested in
sqlTrans = SQLTransformer(statement="""SELECT *, 
lenText(text) AS len,
numWordsText(text) AS numWords,
lenSummary(summary) AS lenS,
numWordsSummary(summary) AS numWordsS,
numExclMark(text) AS numExclMarks,
numExclMarkSummary(summary) AS numExclMarksS
FROM __THIS__""")

In [19]:
# For this simple solution features contain only len(Text)
assembler = VectorAssembler(inputCols=["len", "numWords", "lenS", "numWordsS",
                                      "numExclMarks", "numExclMarksS",
                                      "Score"], outputCol="features")

In [20]:
# Create a classification model based on the decision tree algorithm
dt = DecisionTreeClassifier()

In [21]:
# Define the pipeline that is used to create the logistic regression
# model on the training data.
pipeline = Pipeline().setStages([sqlTrans, assembler,dt])

In [29]:
# We use a ParamGridBuilder to construct a grid of parameter values to 
# search over.
paramGrid = ParamGridBuilder()\
.addGrid(dt.minInstancesPerNode, [1, 10])\
.addGrid(dt.minInfoGain, [0, 0.1])\
.build()

In [30]:
# We now treat the Pipeline as an Estimator, wrapping it in a 
#  CrossValidator instance. This allows us to jointly choose parameters 
# for all Pipeline stages.
# CrossValidator requires 
# - an Estimator
# - a set of Estimator ParamMaps
# - an Evaluator.
cv = CrossValidator()\
.setEstimator(pipeline)\
.setEstimatorParamMaps(paramGrid)\
.setEvaluator(BinaryClassificationEvaluator())\
.setNumFolds(3)

In [31]:
# Fit/Train the model
model = cv.fit(reviews_train)

In [32]:
# Apply the model on the test set
predictions = model.transform(reviews_test).cache()

In [33]:
# Compute statistics
# Accuracy, F1, weighted recall, weighted precision
evaluatorAcc = MulticlassClassificationEvaluator(labelCol="label" , predictionCol= "prediction", metricName = "accuracy")
evaluatorF1 = MulticlassClassificationEvaluator(labelCol="label" , predictionCol= "prediction", metricName = "f1")
evaluatorRecall = MulticlassClassificationEvaluator(labelCol="label" , predictionCol= "prediction", metricName = "weightedRecall")
evaluatorPrecision = MulticlassClassificationEvaluator(labelCol="label" , predictionCol= "prediction", metricName = "weightedPrecision")

print("Accuracy:", evaluatorAcc.evaluate(predictions))
print("F1:", evaluatorF1.evaluate(predictions))
print("Weighted Recall:", evaluatorRecall.evaluate(predictions))
print("Weighted Precision:", evaluatorPrecision.evaluate(predictions))

Accuracy: 0.7278231787900855
F1: 0.7195329239757465
Weighted Recall: 0.7278231787900854
Weighted Precision: 0.7208703237040401


In [34]:
#  Compute the confusion matrix
#                     Predicted  
#  Actual       Useful   Useless
#  Useful          A        B
#  Useless          C        D

A = predictions.filter("prediction=1 and label=1").count()
B = predictions.filter("prediction=0 and label=1").count()
C = predictions.filter("prediction=1 and label=0").count()
D = predictions.filter("prediction=0 and label=0").count()

print("                       Predicted")
print("  Actual \t Useful\tUseless")
print("  Useful \t "+str(A)+ "\t\t"+str(B))
print("  Useless \t "+str(C)+ "\t\t"+str(D))

                       Predicted
  Actual 	 Useful	Useless
  Useful 	 39719		7191
  Useless 	 12992		14252


In [35]:
# Precision and recall for the two classes
# Useful
if A+C==0:
    print("Precision(Useful): undefined")
else:
    print("Precision(Useful):"+str(A/(A+C)))
    
    
print("Recall(Useful):"+str(A/(A+B)))

# Useless 
if B+D==0:
    print("Precision(Useless): undefined")
else:
    print("Precision(Useless):"+str(D/(B+D)))
    
print("Recall(Useless):"+str(D/(C+D)))

Precision(Useful):0.7535239323860294
Recall(Useful):0.8467064591771477
Precision(Useless):0.6646458051578604
Recall(Useless):0.5231243576567317
